In [1]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

import re, math, random, json
import pandas as pd
from tqdm import tqdm
import demoji

from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train_json = './dataset//nikluge-sa-2022-train.jsonl'
# train_raw = pd.read_json(train_json, lines=True)
# train_raw.annotation = train_raw.annotation.apply(lambda x: str(x))
# train_raw = train_raw[['sentence_form', 'annotation']]
# dup = train_raw.duplicated(keep=False)
# train_raw[dup]

In [3]:
# dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
# dev_raw = pd.read_json(dev_json, lines=True)
# dev_raw.annotation = dev_raw.annotation.apply(lambda x: str(x))
# dev_raw = dev_raw[['sentence_form', 'annotation']]
# dup = dev_raw.duplicated(keep=False)
# dev_raw[dup]

In [4]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

In [5]:
train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

In [6]:
# train.iloc[2319]
train = train.drop(2319)
# dev.iloc[1692]
dev = dev.drop(1692)

# Preprocess

## Before

In [7]:
for el in train.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False).sentence_form:
    print(el)

나스 발색력은 원래 알아주기 때문에 좀 더 촉촉한 걸 찾는다면 라나
휴대가 간편한 usb연결 방식이라 캠핑장에서도 사용할수있어 굿굿~♩
끈적임없이 바르는 순간 밀착 되고 수분에센스 71%함유로 수분 팡팡 촉촉함까지
굳...
저의 고민을 해결해주겠는데요^^
기존 궐련형 아이코스, 릴, 글로보다 성능면에서 확실히 우수
당연히 쟁여놔야할 필수템은 NEW 아이소이 잡티세럼Ⅱ 한정세트 ★
사용해보면 진짜 반하는 #워시오프마스크 #sum37
아이가 좋아하는 동물이나 과일도 있어서 더욱 만족스러웠네요.
울 아기 로션으로 계속써야겠어요😍
인정
많은 용량에 한번 반하지만 사용후 느낌이 더 만족스럽다는거 !
곧 다가오는 발렌타인데이 남친 선물 추천!!
유해물질, 화재위험 없이 안전하게 사용 가능한 #아르셰모기퇴치기 ❗
여드름용, 주름용, 화이트닝 등등 이 것 저 것 쳐발쳐발 하던 예전과는 다르게 #피부뿌리부터힘있게 #스킨케어 해주니 강한 #화장품을 찾지 않게 되더라구요~


In [8]:
def textCleaner(text):
    text = re.sub(r'&.+&', '<숨김>', text)
    text = re.sub(r'[^A-Za-z0-9가-힣]', ' ', text)
    
    text = clean(
        text,
        fix_unicode=False,
        to_ascii=False,
        lower=True,
        normalize_whitespace=True,
        no_line_breaks=False,
        strip_lines=True,
        keep_two_line_breaks=False,
        no_urls=False,
        no_emails=False,
        no_phone_numbers=False,
        no_numbers=True,
        no_digits=True,
        no_currency_symbols=False,
        no_punct=True,
        no_emoji=True,
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="0",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        replace_with_punct="",
        lang="en",
    )
    
    # sent = demoji.replace_with_desc(string=sent, sep= " ")
    # sent = re.sub(r'\s+', ' ', sent)
    # sent = re.sub(r'^#$', '', sent)
    
    # text = re.sub(r'[A-Za-z]+', 'alphabet', text)
    # text = re.sub(r'[`~$^+=|><]', '', text)
    # text = re.sub(r'[ㄱ-ㅎ|ㅏ-ㅣ]', '', text)
    
    # text = spacing(text)
    # text = spell_checker.check(text).checked
    return text.upper().strip()

In [9]:
def preprocess(sent):
    sent = textCleaner(sent)
    return sent

In [10]:
train.sentence_form = train.sentence_form.apply(preprocess)
dev.sentence_form = dev.sentence_form.apply(preprocess)
test.sentence_form = test.sentence_form.apply(preprocess)
total = pd.concat([train, dev, test])

Test

In [11]:
case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣 ]+', case=False, flags=0, na=None, regex=True)
for e in total[case].sentence_form:
    print(e)

## After

In [12]:
for el in total.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False).sentence_form:
    print(el)

천연식물로 추출한 천연재료로 항균 소취기능 피부보호기능까지 있어 안심하고 사용하네요
제가 제일 처음에 클럭을 선택한 이유는 디자인도 깔끔하고 다른 것보다 넓은 패드가 있어서 였거든요
사용할수록 속전조함이 해결되고 피부톤도 좋아져서 계속 사용하게 되는 매력
진작알았다면 이유식 시작때부터 요거쓰는건데
숨김 가 조아하는 밀크향이라 양치시간이 즐거워
순하고 좋아서 잘쓰고 있던 누잉화장품
맘에들어 요새 자주 사용하게 되넹
너무 예쁘죠
건성팩트 라던데 수분부족형지성 인 나에게도 굿
애미피부도 소중하니까용
린스 없이도 머릿결이 부들부들해
조금 신기한건 이 샴푸는 꾸덕 하진 않아요
걱정 말고 사용해 보세요
신체의 곡선을 빈틈없이 지지해 주는 센스
근데 이 앰플이 글쎄 고농축 00종의 천연성분 들이 통으로 들어있다고함


# Reformat

In [13]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]

more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

new_tokens = entity_property_pair + more_tokens

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

In [14]:
len(entity_property_pair)

25

In [15]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property not in entity_property_pair:
                    print(row.id, entity_property)
                if sentiment not in polarity_id_to_name:
                    print(row.id, entity_property, sentiment)
                    continue
                if entity_property == pair:
                    ep.append([id, utterance, entity_property, tf_name_to_id['True']])
                    p.append([id, utterance, entity_property, sentiment])
                    isPairInOpinion = True
                    break

            if isPairInOpinion is False:
                ep.append([id, utterance, pair, tf_name_to_id['False']])

    return ep, p

In [16]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [17]:
ep_train, p_train = reformat(train)
ep_train = pd.DataFrame(ep_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_train = pd.DataFrame(p_train, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

ep_dev, p_dev = reformat(dev)
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

In [18]:
### more than 1 entitiy in a single row
# condition = train.annotation.apply(lambda x: len(x)) > 1
# print(train.annotation[condition].count())
# print(train.annotation[condition].apply(lambda x: len(x)).sum())
# 489-235

In [19]:
p_binary_train = reformat_p_binary(p_train)
p_binary_train = pd.DataFrame(p_binary_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [20]:
p_binary_train

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#품질#positive,1
1,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#품질#negative,0
2,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#품질#neutral,1
3,nikluge-sa-2022-train-00002,이거 뭐 삐꾸를 준 거 아냐 불안하고 거금 투자한 게 왜 이래 싶어서 정이 확 떨어...,본품#품질#positive,1
4,nikluge-sa-2022-train-00002,이거 뭐 삐꾸를 준 거 아냐 불안하고 거금 투자한 게 왜 이래 싶어서 정이 확 떨어...,본품#품질#negative,0
...,...,...,...,...
9583,nikluge-sa-2022-train-03000,마지막으로 귀여워서 집어온 모자,제품 전체#디자인#negative,1
9584,nikluge-sa-2022-train-03000,마지막으로 귀여워서 집어온 모자,제품 전체#디자인#neutral,1
9585,nikluge-sa-2022-train-03001,일본유니클로 사랑해,브랜드#일반#positive,0
9586,nikluge-sa-2022-train-03001,일본유니클로 사랑해,브랜드#일반#negative,1


In [21]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [22]:
p_binary_dev

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#positive,0
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#negative,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#neutral,1
3,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데 샤워는 자주 해도 목욕은 그렇게 자주가 아님 이것도 약재는...,본품#일반#positive,0
4,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데 샤워는 자주 해도 목욕은 그렇게 자주가 아님 이것도 약재는...,본품#일반#negative,1
...,...,...,...,...
9001,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음,제품 전체#일반#negative,1
9002,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음,제품 전체#일반#neutral,1
9003,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,제품 전체#품질#positive,0
9004,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,제품 전체#품질#negative,1


# Counting

In [35]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
p_binary_train = p_binary_train.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
print('\nafter drop_duplicates')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))

binary_multi: 75000 69825 3196 3002
binary_binary: 75000 69825 9588 9006

after drop_duplicates
binary_multi: 75000 69825 3196 3002
binary_binary: 75000 69825 9588 9006


(None, None)

In [36]:
save_path = './dataset/cleaned_v1'

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)
p_train.to_csv(f'{save_path}/pc_train.csv', index=False)
p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False)
p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

# ASC Augmentation

In [ ]:
model_checkpoint = '/content/drive/MyDrive/aspect_based_sentiment_analysis/base_model/klue_roberta_base/v2/klue_roberta_base_mlm/checkpoint-19860'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
sTokens = tokenizer.all_special_tokens

def delTokens(sent):
    sent = sent.split(' ')
    temp = []
    for e in sent:
        if e not in sTokens:
            temp.append(e)
    return ' '.join(temp)

In [ ]:
positive, negative, neutral = p_train[p_train.sentiment == 'positive'], p_train[p_train.sentiment == 'negative'], p_train[p_train.sentiment == 'neutral']

In [ ]:
len(positive), len(negative), len(neutral)

(3043, 58, 95)

In [ ]:
(58 * 3) * 4 * 3, (95 * 3) * 4 * 2 # bt ri rr

(2088, 2280)

Back Translation / Random Insertion / Random Replacement / Random Swap / Random Deletion

In [ ]:
def backTrans(text):
    aug1 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='en')
    aug1 = ts.papago(aug1, sleep_seconds=5, from_language='en', to_language='ko')

    aug2 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='ja')
    aug2 = ts.papago(aug2, sleep_seconds=5, from_language='ja', to_language='ko')

    return [aug1, aug2]

def randomInsert(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomReplace(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomSwap(num, sample):
    aug = naw.RandomWordAug(action='swap', aug_min=1, aug_max=1, aug_p=0.3)    
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

def randomSplit(num, sample):
    aug = naw.SplitAug(aug_min=1, aug_max=1, aug_p=0.3, min_char=3)
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

In [ ]:
(58 * 3) * 5 * 4, (95 * 3) * 4 * 3 # bt ri rr

(3480, 3420)

In [ ]:
def backtransRoutine(data2augment, output_path):
    print('back translation started.')
    temp = []
    for row in data2augment:
        augs = backTrans(row[1])
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(len(f'back translation finished.\ncurrent count: {len(data2augment)}'))

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
import os

def edaRoutine(data2augment, ri, rr, output_path):
    print(f'current count: {len(data2augment)}')
    print('random insertion started.')
    temp = []
    for row in data2augment:
        augs = randomInsert(ri, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random insertion finished.\ncurrent count: {len(data2augment)}')

    print('random replacement started.')
    temp = []
    for row in data2augment:
        augs = randomReplace(rr, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random replacement finished.\ncurrent count: {len(data2augment)}')

    print('random swap and split started.')
    while len(data2augment) < len(positive):
        temp = []
        k = random.randrange(len(negative))
        id, text, entity, sentiment = data2augment[k]

        selector = random.randint(0,1)
        if selector == 0:
            augs = randomSwap(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        else:
            augs = randomSplit(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        if len(data2augment)%25 == 0:
            print(f'random swap and split in progress.\ncurrent count: {len(data2augment)}')

    print(f'whole augmentation routine finished.\ntotal count: {len(data2augment)}')

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
### negative
# # back translation

# data2augment = negative.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_bt.csv'
output_path = os.path.join(output_folder, output_file)

# negative_bt = backtransRoutine(data2augment, output_path)
negative_bt = pd.read_csv(output_path)
negative_bt = negative_bt.values.tolist()
# RI / RR

ri = 4 # times - 1
rr = 3 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_aug.csv'
output_path = os.path.join(output_folder, output_file)

# negative_aug = edaRoutine(negative_bt, ri, rr, output_path)
negative_aug = pd.read_csv(output_path)

In [ ]:
negative_aug
negative_aug = negative_aug.drop_duplicates()

In [ ]:
# negative_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# negative_aug.sort_values('id').head(50).sentence_form

In [ ]:
### neutral
# back translation

# data2augment = neutral.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_bt.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_bt = backtransRoutine(data2augment, output_path)
neutral_bt = pd.read_csv(output_path)
neutral_bt = neutral_bt.values.tolist()

# RI / RR

ri = 3 # times - 1
rr = 2 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_aug.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_aug = edaRoutine(neutral_bt, ri, rr, output_path)
neutral_aug = pd.read_csv(output_path)

In [ ]:
neutral_aug
neutral_aug = neutral_aug.drop_duplicates()

In [ ]:
# neutral_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# neutral_aug.sort_values('id').head(50).sentence_form

In [ ]:
p_train_aug = pd.concat([positive, negative_aug, neutral_aug])

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [ ]:
p_binary_train_aug = reformat_p_binary(p_train_aug)
p_binary_train_aug = pd.DataFrame(p_binary_train_aug, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train_aug

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#positive,0
1,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#negative,1
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#neutral,1
3,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#positive,0
4,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#negative,1
...,...,...,...,...
27316,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#negative,1
27317,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#neutral,0
27318,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#positive,1
27319,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#negative,1


In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#positive,0
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#negative,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#neutral,1
3,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#positive,0
4,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#negative,1
...,...,...,...,...
9001,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#negative,1
9002,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#neutral,1
9003,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#positive,0
9004,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#negative,1


# Counting

In [ ]:
len(ep_train), len(ep_dev), len(p_train), len(p_dev)

(75000, 69825, 3196, 3002)

In [ ]:
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

(75000, 69825, 27321, 9006)

In [ ]:
ep_train = ep_train.drop_duplicates()
p_binary_train_aug = p_binary_train_aug.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

(75000, 69825, 27321, 9006)

# Export

In [ ]:
%cd /content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv('ce_train.csv', index=False)
p_binary_train_aug.to_csv('pc_binary_train_aug.csv', index=False)
ep_dev.to_csv('ce_dev.csv', index=False)
p_binary_dev.to_csv('pc_binary_dev.csv', index=False)

/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11


In [ ]:
# emojis = pd.concat([ep_train.sentence_form, p_train.sentence_form, ep_dev.sentence_form, p_dev.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
# emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))

In [ ]:
df = pd.read_csv('ce_train.csv')
df[df.id == 'nikluge-sa-2022-train-00065']

,id,sentence_form,entity_property,labels
1600,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#가격,1
1601,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#다양성,1
1602,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#디자인,1
1603,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#인지도,1
1604,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#일반,1
1605,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#편의성,1
1606,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#품질,1
1607,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,브랜드#가격,1
1608,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,브랜드#디자인,1
1609,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,브랜드#인지도,1


In [ ]:
df = pd.read_csv('ce_dev.csv')
df

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#가격,1
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#다양성,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#디자인,1
3,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#인지도,1
4,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#일반,1
...,...,...,...,...
69820,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#다양성,1
69821,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#디자인,1
69822,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#일반,1
69823,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#편의성,1


In [ ]:
df = pd.read_csv('pc_binary_train_aug.csv')
df

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#positive,0
1,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#negative,1
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#neutral,1
3,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#positive,0
4,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#negative,1
...,...,...,...,...
27316,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#negative,1
27317,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#neutral,0
27318,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#positive,1
27319,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#negative,1


In [ ]:
df = pd.read_csv('pc_binary_dev.csv')
df

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#positive,0
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#negative,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#neutral,1
3,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#positive,0
4,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#negative,1
...,...,...,...,...
9001,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#negative,1
9002,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#neutral,1
9003,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#positive,0
9004,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#negative,1
